In [37]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


In [92]:
spect = []
spect_id = []
for j in ['MJF','Letterman']:
    file = []
    for root, dirs, files in os.walk('spectrograms/'+j):
        file += glob.glob(os.path.join(root,'*.png'))
    for i in file:
        if j == 'Letterman':
            x = plt.imread('spectrograms/'+j+'/'+i[22:]) 
        if j == 'MJF':
            x = plt.imread('spectrograms/'+j+'/'+i[17:])
        spect.append(x)
        spect_id.append(j)
    
x_train = spect
y_train = spect_id

In [93]:
#create subset of data for test and train
import random
print(len(x_train))
test = 140*.2
test_indices = random.sample(range(len(x_train)), 28)
train_indices = []
for i in range(len(x_train)):
    if i not in test_indices:
        train_indices.append(i)

x_new_train = []
y_new_train = []
x_test = []
y_test = []

for i in test_indices:
    x_test.append(x_train[i])
    y_test.append(y_train[i])


for i in train_indices:
    x_new_train.append(x_train[i])
    y_new_train.append(y_train[i])

157


In [96]:
print((y_test))

['Letterman', 'Letterman', 'MJF', 'Letterman', 'Letterman', 'Letterman', 'Letterman', 'MJF', 'Letterman', 'MJF', 'Letterman', 'Letterman', 'Letterman', 'MJF', 'MJF', 'MJF', 'Letterman', 'Letterman', 'MJF', 'Letterman', 'MJF', 'MJF', 'MJF', 'Letterman', 'MJF', 'MJF', 'MJF', 'Letterman']


In [97]:
encoder = LabelEncoder()
y_temp_train = y_new_train
encoder.fit(y_temp_train)
encoded_Y = encoder.transform(y_temp_train)
dummy_y = np_utils.to_categorical(encoded_Y)

y_temp2_train = y_test
encoder.fit(y_temp2_train)
encoded_Y = encoder.transform(y_temp2_train)
dummy2_y = np_utils.to_categorical(encoded_Y)

In [98]:
from keras import backend as K

In [99]:
K.image_data_format()

'channels_last'

In [100]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras import optimizers
from keras.layers.advanced_activations import ELU, PReLU, LeakyReLU
from keras.layers import Dense, Dropout, Activation, Flatten

In [101]:
model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(288, 432, 4)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(10))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 288, 432, 8)       296       
_________________________________________________________________
activation_31 (Activation)   (None, 288, 432, 8)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 286, 430, 8)       584       
_________________________________________________________________
activation_32 (Activation)   (None, 286, 430, 8)       0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 143, 215, 8)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 143, 215, 8)       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 143, 215, 16)      1168      
__________

In [102]:
model.fit(np.array(x_new_train), dummy_y,
              batch_size=30,
              epochs=100,
              verbose=1, 
             validation_data=(np.array(x_test[:1000]), dummy2_y[:1000]))
model.save_weights('my_modelv2_weights.h5')

Train on 129 samples, validate on 28 samples
Epoch 1/100
129/129 [==============================] - 17s 131ms/step - loss: 1.1076 - acc: 0.5194 - val_loss: 0.6894 - val_acc: 0.6071
Epoch 2/100
129/129 [==============================] - 14s 107ms/step - loss: 0.9376 - acc: 0.4961 - val_loss: 0.8109 - val_acc: 0.4643
Epoch 3/100
129/129 [==============================] - 13s 102ms/step - loss: 0.9019 - acc: 0.4419 - val_loss: 0.6883 - val_acc: 0.5357
Epoch 4/100
129/129 [==============================] - 13s 102ms/step - loss: 0.8098 - acc: 0.3953 - val_loss: 0.6931 - val_acc: 0.5357
Epoch 5/100
129/129 [==============================] - 13s 102ms/step - loss: 0.7457 - acc: 0.4651 - val_loss: 0.6916 - val_acc: 0.4643
Epoch 6/100
129/129 [==============================] - 14s 107ms/step - loss: 0.7185 - acc: 0.4806 - val_loss: 0.7961 - val_acc: 0.4643
Epoch 7/100
129/129 [==============================] - 23s 177ms/step - loss: 0.7775 - acc: 0.5039 - val_loss: 0.6791 - val_acc: 0.5357
Epo

129/129 [==============================] - 15s 120ms/step - loss: 0.0531 - acc: 1.0000 - val_loss: 0.3710 - val_acc: 0.8214
Epoch 61/100
129/129 [==============================] - 15s 118ms/step - loss: 0.0310 - acc: 1.0000 - val_loss: 0.3753 - val_acc: 0.8214
Epoch 62/100
129/129 [==============================] - 15s 120ms/step - loss: 0.0235 - acc: 1.0000 - val_loss: 0.3663 - val_acc: 0.8214
Epoch 63/100
129/129 [==============================] - 16s 123ms/step - loss: 0.0190 - acc: 1.0000 - val_loss: 0.3664 - val_acc: 0.8214
Epoch 64/100
129/129 [==============================] - 16s 120ms/step - loss: 0.0204 - acc: 1.0000 - val_loss: 0.4420 - val_acc: 0.8214
Epoch 65/100
129/129 [==============================] - 15s 119ms/step - loss: 0.1048 - acc: 0.9612 - val_loss: 0.4448 - val_acc: 0.8214
Epoch 66/100
129/129 [==============================] - 15s 120ms/step - loss: 0.0179 - acc: 1.0000 - val_loss: 0.3497 - val_acc: 0.8214
Epoch 67/100
129/129 [==============================] 

In [50]:
y_test


['MJF',
 'Letterman',
 'MJF',
 'MJF',
 'Letterman',
 'Letterman',
 'Letterman',
 'Letterman',
 'Letterman',
 'Letterman',
 'MJF',
 'MJF',
 'Letterman',
 'Letterman',
 'MJF',
 'MJF',
 'Letterman',
 'MJF',
 'Letterman',
 'Letterman',
 'MJF',
 'MJF',
 'Letterman',
 'Letterman',
 'Letterman',
 'Letterman',
 'Letterman',
 'Letterman']

In [106]:
r = 20
print(model.predict_classes(np.expand_dims(np.array(x_test[r]), axis=0)))
print(dummy2_y)

[1]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [107]:
model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(288, 432, 4)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(10))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.load_weights('my_modelv2_weights.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 288, 432, 8)       296       
_________________________________________________________________
activation_37 (Activation)   (None, 288, 432, 8)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 286, 430, 8)       584       
_________________________________________________________________
activation_38 (Activation)   (None, 286, 430, 8)       0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 143, 215, 8)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 143, 215, 8)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 143, 215, 16)      1168      
__________

In [108]:
from keras.models import Model
#flatten_5 might need to be changed depending on the model
model2 = Model(inputs=model.input, outputs=model.get_layer('flatten_7').output)
svm_x_train = []
svm_y_train = []
for i in range(len(x_new_train)):
    x_1 = np.expand_dims(x_new_train[i], axis=0)
    flatten_2_features = model2.predict(x_1)
    svm_x_train.append(flatten_2_features)
    svm_y_train.append(dummy_y[i])

In [87]:
svm_y_train

[array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0.

In [109]:
from sklearn import svm
svm_x_train = np.array(svm_x_train)
clf = svm.SVC(kernel='rbf', class_weight='balanced')
dataset_size = len(svm_x_train)
svm_x_train = np.array(svm_x_train).reshape(dataset_size,-1)
svm_y_train = np.array(svm_y_train)
svm_y_train = [np.where(r==1)[0][0] for r in svm_y_train]

In [110]:
clf.fit(svm_x_train, svm_y_train)

/Users/bryanmckinney/miniconda3/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [111]:
svm_x_test = []
svm_y_test = []
for i in range(len(x_test)):
    x_1 = np.expand_dims(x_test[i], axis=0)
    #x_1 = preprocess_input(x_1)
    flatten_2_features = model2.predict(x_1)
    svm_x_test.append(flatten_2_features)
    svm_y_test.append(dummy2_y[i])
svm_x_test = np.array(svm_x_test)

In [112]:
dataset_size = len(svm_x_test)
svm_x_test = np.array(svm_x_test).reshape(dataset_size,-1)
svm_y_test = [np.where(r==1)[0][0] for r in svm_y_test]
print(svm_y_test)
print(clf.predict(svm_x_test))
from sklearn.metrics import accuracy_score
accuracy_score(svm_y_test, clf.predict(svm_x_test))

[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


0.4642857142857143